### Approach One (Without preemptive resource)

In [2]:
!pip install simpy

In [3]:
import random
import simpy

In [8]:


def coffee_machine(env):
  while True:
    print(f"Coffee machine ready to use at {env.now: .2f}.")
    random_interrarrival_time = 100
    yield env.timeout(random_interrarrival_time)
    print(f"Coffee over! Call worker at {env.now: .2f}.")
    cleaning.interrupt()
    global refilled_event
    yield refilled_event
    refilled_event = env.event()

def clean_floor(env, worker):
  time_to_clean = 120
  while True:
    try:
      while time_to_clean:
        print(f"Cleaning started at {env.now: .2f} and time to clean = {time_to_clean :.2f}.")
        with worker.request() as req:
          yield req
          start = env.now
          yield env.timeout(time_to_clean)
          time_to_clean = 120
    except simpy.Interrupt:
      current_time = env.now
      time_to_clean -= current_time - start
      if time_to_clean == 0:
        time_to_clean = 120
      print(f"Cleaning process interrupted at {env.now:.2f} and time left to clean = {time_to_clean:.2f}.")
      yield env.process(refilling(env, worker))


def refilling(env, worker):

  with worker.request() as req:
    yield req
    print(f"Refilling process started at time:{env.now:.2f}.")
    yield env.timeout(15)
    print(f"Refilled. Machine ready to use at {env.now :.2f}.")
    global refilled_event
    refilled_event.succeed()

env = simpy.Environment()
refilled_event = env.event()
worker = simpy.Resource(env, capacity=1)
env.process(coffee_machine(env))
cleaning = env.process(clean_floor(env, worker))
env.run(until = 24*60)

Coffee machine ready to use at  0.00.
Cleaning started at  0.00 and time to clean = 120.00.
Coffee over! Call worker at  100.00.
Cleaning process interrupted at 100.00 and time left to clean = 20.00.
Refilling process started at time:100.00.
Refilled. Machine ready to use at 115.00.
Coffee machine ready to use at  115.00.
Cleaning started at  115.00 and time to clean = 20.00.
Cleaning started at  135.00 and time to clean = 120.00.
Coffee over! Call worker at  215.00.
Cleaning process interrupted at 215.00 and time left to clean = 40.00.
Refilling process started at time:215.00.
Refilled. Machine ready to use at 230.00.
Coffee machine ready to use at  230.00.
Cleaning started at  230.00 and time to clean = 40.00.
Cleaning started at  270.00 and time to clean = 120.00.
Coffee over! Call worker at  330.00.
Cleaning process interrupted at 330.00 and time left to clean = 60.00.
Refilling process started at time:330.00.
Refilled. Machine ready to use at 345.00.
Coffee machine ready to use at

### Approach 2 ( Preemptive Resource)

In [12]:


def coffee_machine(env):
  while True:
    print(f"Coffee machine ready to use at {env.now: .2f}.")
    random_interrarrival_time = 100
    yield env.timeout(random_interrarrival_time)
    print(f"Coffee over! Call worker at {env.now: .2f}.")
    yield env.process(refilling(env, worker))


def clean_floor(env, worker):
  time_to_clean = 120
  while True:
    try:
      while time_to_clean:
        print(f"Cleaning started at {env.now: .2f} and time to clean = {time_to_clean :.2f}.")
        with worker.request(priority = 2) as req:
          yield req
          start = env.now
          yield env.timeout(time_to_clean)
          print(f"Cleaning finished at {env.now: .2f}.")
          time_to_clean = 120
    except simpy.Interrupt:
      current_time = env.now
      time_to_clean -= current_time - start
      if time_to_clean == 0:
        time_to_clean = 120
      print(f"Cleaning process interrupted at {env.now:.2f} and time left to clean = {time_to_clean:.2f}.")


def refilling(env, worker):

  with worker.request(priority=1) as req:
    yield req
    print(f"Refilling process started at time:{env.now:.2f}.")
    yield env.timeout(15)
    print(f"Refilled. Machine ready to use at {env.now :.2f}.")


env = simpy.Environment()
worker = simpy.PreemptiveResource(env, capacity=1)
env.process(coffee_machine(env))
cleaning = env.process(clean_floor(env, worker))
env.run(until = 24*60)

Coffee machine ready to use at  0.00.
Cleaning started at  0.00 and time to clean = 120.00.
Coffee over! Call worker at  100.00.
Cleaning process interrupted at 100.00 and time left to clean = 20.00.
Cleaning started at  100.00 and time to clean = 20.00.
Refilling process started at time:100.00.
Refilled. Machine ready to use at 115.00.
Coffee machine ready to use at  115.00.
Cleaning finished at  135.00.
Cleaning started at  135.00 and time to clean = 120.00.
Coffee over! Call worker at  215.00.
Cleaning process interrupted at 215.00 and time left to clean = 40.00.
Cleaning started at  215.00 and time to clean = 40.00.
Refilling process started at time:215.00.
Refilled. Machine ready to use at 230.00.
Coffee machine ready to use at  230.00.
Cleaning finished at  270.00.
Cleaning started at  270.00 and time to clean = 120.00.
Coffee over! Call worker at  330.00.
Cleaning process interrupted at 330.00 and time left to clean = 60.00.
Cleaning started at  330.00 and time to clean = 60.00.